In [1]:
import csv
import json
from lxml import etree
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

# récupérer les annotations manuelles sur un fichier au format "Inline XML"
def recup_annotations_manuelles(fichier):
    annotations = []
    parser = etree.XMLParser(recover=True)
    root = etree.parse(fichier, parser)
    balise_EN = "de.tudarmstadt.ukp.dkpro.core.api.ner.type.NamedEntity"
    
    for el in root.iter(balise_EN):
        annotations.append((int(el.attrib["begin"]), int(el.attrib["end"]), el.attrib["value"]))
    return annotations

# récupérer les annotations et le texte à partir d'un fichier au format json particulier (exporter format UIMA CAS JSON sur INCEPTION)
def recup_annotations_texte(fichier):
    dico = lire_json(fichier_texte_ann_manuelles)
    annots = dico["_views"]["_InitialView"]["NamedEntity"]
    annotations = []
    for ann in annots:
        annotations.append((ann["begin"], ann["end"], ann["value"]))
    texte = dico["_referenced_fss"]["1"]["sofaString"]
    return annotations, texte

# récupérer le texte d'un fichier ".txt"
def recup_texte(fichier):
    with open(fichier, "r", encoding="utf-8") as f:
        texte = f.read()
    return texte

# récupérer les annotations d'un fichier csv avec les entêtes "Indice_Début", "Indice Fin", "Tag"
def recup_annotations_csv(fichier):
    annotations = []
    with open(fichier, "r", encoding="utf-8", newline="") as csvfile:
        reader = csv.DictReader(csvfile, delimiter=",")
        for row in reader:
            #print(row['Tag'], row['Indice_Début'], row['Indice_Fin'])
            annotations.append((int(row['Indice_Début']), int(row['Indice_Fin']), row['Tag']))

    return annotations

# lecture d'un fichier json à partir de son chemin
# renvoie le dictionnaire correspondant
def lire_json(fichier):
    with open(fichier, "r", encoding="utf-8") as fin:
        dic = json.load(fin)
    return dic

# fonction d'évaluation pour Spacy
# prend en argument le model utilisé et les donnees à tester, sous la forme d'une liste de tuples (texte, annotations attendues)
def evaluate(ner_model, donnees):
    #scorer = Scorer()
    for input_, annot in donnees:
        scorer = Scorer()
        doc_gold_text = ner_model.make_doc(input_)
        # Gold : valeurs de référence
        gold = GoldParse(doc_gold_text, entities=annot)
        # Valeurs prédites par le modèle
        pred_value = ner_model(input_)
        #print(pred_value)
        #for ent in pred_value.ents:
            #print(ent.text, ent.start_char, ent.end_char, ent.label_)
        scorer.score(pred_value, gold)
    return scorer.scores

In [2]:
chemin_fichier = "Evaluation/Fichiers/5408048"
fichier_texte_ann_manuelles = "%s_texte_manuelles.json" % chemin_fichier

fichier_text_corr = "%s_correction.txt" % chemin_fichier

fichier_an_auto = "%s_annotations.csv" % chemin_fichier
fichier_an_auto_corr = "%s_annotations_correction.csv" % chemin_fichier


manuelles, texte = recup_annotations_texte(fichier_texte_ann_manuelles)

texte_corr = recup_texte(fichier_text_corr)
print(len(texte_corr))
auto = recup_annotations_csv(fichier_an_auto)
print(len(auto))
auto_corr = recup_annotations_csv(fichier_an_auto_corr)
print(len(auto_corr))

96206
224
227


In [7]:
versions = [
    (texte2, manuelles)
]

In [8]:
ner_model = spacy.load('fr_core_news_sm')
results = evaluate(ner_model, versions)
print(results["ents_per_type"])
print("-"*10)

{}
----------


In [6]:
test = "Evaluation/Fichiers/5408048_correction.txt"

with open(test, "r", encoding="utf-8") as fin:
    texte2 = fin.read()

In [11]:
print(auto_corr[:10])
print(auto[:10])

[(100, 104, 'PER'), (110, 112, 'PER'), (116, 123, 'ORG'), (127, 144, 'ORG'), (223, 233, 'ORG'), (258, 271, 'PER'), (307, 316, 'PER'), (318, 326, 'PER'), (330, 369, 'LOC'), (717, 722, 'LOC')]
[(35, 38, 'ORG'), (110, 112, 'LOC'), (116, 123, 'ORG'), (127, 144, 'ORG'), (223, 233, 'ORG'), (255, 263, 'PER'), (306, 315, 'PER'), (317, 325, 'PER'), (329, 368, 'LOC'), (716, 721, 'LOC')]


In [12]:
"jeee m'appelle Nico et je viens de Pariss"

"je m'appelle Nico et je viens de Paris"

"je m'appelle Nico et je viens de Paris"

In [26]:
json_manuelles = []

for annot in manuelles:
    l = list(annot)
    l.append(texte[annot[0]:annot[1]])
    json_manuelles.append(l)


In [30]:
import json

with open("annotations_manuelles.json", "w", encoding="utf-8") as fout:
    fout.write(json.dumps(json_manuelles, indent=2, ensure_ascii=False))

In [41]:
json_spacy = []

for annot in auto:
    l = list(annot)
    l.append(texte[annot[0]:annot[1]])
    json_spacy.append(l)
    

with open("annotations_spacy.json", "w", encoding="utf-8") as fout:
    fout.write(json.dumps(json_spacy, indent=2, ensure_ascii=False))

In [42]:
json_spacy_correction = []

for annot in auto_corr:
    l = list(annot)
    l.append(texte[annot[0]:annot[1]])
    json_spacy_correction.append(l)
    
with open("annotations_spacy_corrige.json", "w", encoding="utf-8") as fout:
    fout.write(json.dumps(json_spacy_correction, indent=2, ensure_ascii=False))